In [3]:
import pandas as pd
import numpy as np
import pickle
from fbprophet import Prophet
import urllib
import urllib.parse
import json
import plotly.graph_objects as go
from urllib.request import urlopen
from weather import *
import dropbox
from dropboxUtils import *
import pandas as pd
import os
import argparse
import contextlib
import datetime
import os
import six
import sys
import time
import unicodedata
from datetime import *
from wwo_hist import retrieve_hist_data
import pytz

In [4]:
weather_api_key = 'd26ec461fcd64cc583b134233201002'
dropbox_api_key = 'wJzLu96m0qIAAAAAAAHPFR8xX-NLBJyqa7QSqSe4Uix8ZzEN6uiYaOyyQLUR6GYJ'
dbx = dropbox.Dropbox(dropbox_api_key)
dbx.users_get_current_account()
print('connected to dropbox')

INFO:dropbox:Request to users/get_current_account


connected to dropbox


In [5]:
retrieve_future_data(api_key=weather_api_key,
                     location_list=['Montreal'],
                     frequency=24, num_of_days=14,
                     location_label=False,
                     export_csv=True,
                     store_df=False)

[]

In [6]:
weather_forecast = pd.read_csv('Montreal-daily.csv')
weather_forecast['ds'] = pd.to_datetime(weather_forecast['ds'])

In [7]:
weather_forecast

,ds,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,uvIndex.1,moon_illumination,DewPointC,FeelsLikeC,...,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2020-02-09,-9,-23,0.0,7.1,1,1,100,-17,-20,...,-20,10,56,92,0.1,1032,-9,6,139,6
1,2020-02-10,0,-6,9.4,3.8,1,1,83,-3,-5,...,-5,15,90,96,11.5,1017,0,5,222,9
2,2020-02-11,0,-7,0.3,3.8,1,1,75,-4,-6,...,-6,13,96,94,0.3,1014,0,7,203,8
3,2020-02-12,-1,-7,0.3,5.4,1,1,68,-6,-10,...,-10,30,45,87,0.1,1014,-1,10,239,18
4,2020-02-13,-1,-15,0.0,0.0,1,1,60,-8,-12,...,-12,16,100,95,9.4,1004,-1,3,70,11
5,2020-02-14,-20,-27,0.0,0.0,1,1,53,-25,-31,...,-31,17,59,83,0.8,1030,-20,9,304,10
6,2020-02-15,-15,-30,0.0,0.0,11,11,45,-23,-28,...,-28,11,57,89,0.0,1038,-15,10,209,7
7,2020-02-16,1,-10,0.0,0.0,11,11,38,-5,-10,...,-10,28,80,92,7.3,1018,1,6,206,17
8,2020-02-17,-1,-8,0.0,0.0,11,11,30,-4,-7,...,-7,15,78,95,0.2,1025,-1,6,170,9
9,2020-02-18,2,-7,0.0,0.0,11,11,22,-2,-7,...,-7,25,100,98,19.7,1013,2,6,97,13


In [8]:
# dl=1 is important
pickle_url = 'https://www.dropbox.com/s/ipj203q24sk0etp/jgh-prophet-daily.pkl?dl=1'
import urllib.request
u = urllib.request.urlopen(pickle_url)
data = u.read()
u.close()

In [9]:
with open('jgh-prophet-daily.pkl', "wb") as f:
    f.write(data)

with open('jgh-prophet-daily.pkl', 'rb') as f:
    m = pickle.load(f)

In [10]:
forecast = m.predict(weather_forecast)

date_string = datetime.now().date().strftime("%Y-%m-%d")
new_predictions = forecast[['ds', 'yhat']]
new_predictions.yhat = new_predictions.yhat.astype('int64')
new_predictions = new_predictions.rename(columns={'yhat': date_string})
new_predictions = new_predictions.set_index('ds')
new_predictions = new_predictions.T

/anaconda3/envs/prophet/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
old_predictions = pd.read_csv(
    'https://www.dropbox.com/s/8v2tnadtjpd5hht/jghPredictions.csv?dl=1', index_col=0)
old_predictions.index = pd.to_datetime(old_predictions.index)
old_predictions.columns = pd.to_datetime(old_predictions.columns)

predictions = pd.concat([old_predictions, new_predictions])
predictions.to_csv('jghPredictions.csv')

In [12]:
upload(dbx, 'jghPredictions.csv', '', '',
       'jghPredictions.csv', overwrite=True)

INFO:dropbox:Request to files/upload


uploaded as b'jghPredictions.csv'


FileMetadata(name='jghPredictions.csv', id='id:HRQT88r_1DgAAAAAAAE2Vg', client_modified=datetime.datetime(2020, 2, 10, 14, 11, 55), server_modified=datetime.datetime(2020, 2, 10, 14, 11, 58), rev='0159e395287e14200000001a109f300', size=1134, path_lower='/jghpredictions.csv', path_display='/jghPredictions.csv', parent_shared_folder_id=None, media_info=None, symlink_info=None, sharing_info=None, is_downloadable=True, export_info=None, property_groups=None, has_explicit_shared_members=None, content_hash='eaee80f42ecb45aadcaadcf63f372409b67bc5b423989349048010690f849b97')